In [1]:
from transfer_learning_publication.data import CaravanDataSource
from transfer_learning_publication.transforms import PipelineBuilder, Log, ZScore
from transfer_learning_publication.containers import TimeSeriesCollection

In [2]:
caravan = CaravanDataSource(
    base_path="/Users/cooper/Desktop/CARAVAN_CLEAN/train",
    region="tajikkyrgyz"
)

basins = caravan.list_gauge_ids()

ts_data = caravan.get_timeseries(
    gauge_ids=basins[:2],
    variables=["streamflow", "total_precipitation_sum"],
)

static_data = caravan.get_static_attributes(
    gauge_ids=basins[:2],
    columns=["area"],
)

In [3]:
static_data.collect()

REGION_NAME,attribute_type,gauge_id,area
str,str,str,f64
"""tajikkyrgyz""","""other""","""tajikkyrgyz_15013""",254.78646
"""tajikkyrgyz""","""other""","""tajikkyrgyz_15016""",330.553623
"""tajikkyrgyz""","""caravan""","""tajikkyrgyz_15013""",null
"""tajikkyrgyz""","""caravan""","""tajikkyrgyz_15016""",null
"""tajikkyrgyz""","""hydroatlas""","""tajikkyrgyz_15013""",null
"""tajikkyrgyz""","""hydroatlas""","""tajikkyrgyz_15016""",null


In [6]:
static_collection = caravan.to_static_attribute_collection(static_data)
static_collection._group_tensors

{'tajikkyrgyz_15013': tensor([254.7865]),
 'tajikkyrgyz_15016': tensor([330.5536])}

In [5]:
ts_collection.feature_names

NameError: name 'ts_collection' is not defined

In [ ]:
ts_collection._group_tensors

In [ ]:
type(ts_collection)

In [ ]:
preprocessing_pipelines = (
    PipelineBuilder(group_identifier="gauge_id")
    .add_per_basin(Log(), columns=["streamflow"])
    .add_global(ZScore(), columns=["streamflow", "total_precipitation_sum"])
    .build()
)

data_transformed = preprocessing_pipelines.fit_transform(ts_data)
data_inv = preprocessing_pipelines.inverse_transform(data_transformed)

In [ ]:
ts_data.head(), data_transformed.head(), data_inv.head()

In [ ]:
caravan.write_timeseries(
    data_transformed,
    "/Users/cooper/Desktop/transfer-learning-publication/data",
    True
)